In [1]:
import pandas as pd
from collections import OrderedDict
import pytz

data = OrderedDict()

data['BTC'] = pd.read_csv("BTC-USD.csv")

data['BTC']['date'] = pd.to_datetime(data['BTC']['time'], unit = 's', utc = True)
data['BTC'].set_index('date', inplace=True)
data['BTC'].drop('time', axis=1, inplace=True)
# 1분 단위 평균으로 바꿔주기 : 현재 거의 1분단위이지만 혹시 에러를 방지하기 위해서
data['BTC'] = data['BTC'].resample("1min").mean()
data['BTC'].fillna(method='ffill', inplace=True)
data['BTC'] = data['BTC'][["low","high","open","close","volume"]]
print(data['BTC'].head())

panel = pd.Panel(data)
panel.minor_axis = ["low","high","open","close","volume"]
panel.major_axis = panel.major_axis.tz_convert(pytz.utc)
print(panel)


                                   low         high         open        close  \
date                                                                            
2018-02-05 17:48:00+00:00  7170.000000  7171.000000  7170.000000  7170.990234   
2018-02-05 17:49:00+00:00  7131.990234  7171.000000  7170.990234  7131.990234   
2018-02-05 17:50:00+00:00  7120.000000  7137.359863  7132.000000  7120.020020   
2018-02-05 17:51:00+00:00  7113.000000  7121.000000  7120.040039  7113.000000   
2018-02-05 17:52:00+00:00  7113.000000  7122.000000  7113.000000  7121.990234   

                              volume  
date                                  
2018-02-05 17:48:00+00:00   3.425961  
2018-02-05 17:49:00+00:00   5.209975  
2018-02-05 17:50:00+00:00  14.767619  
2018-02-05 17:51:00+00:00  18.237879  
2018-02-05 17:52:00+00:00  22.768671  
<class 'pandas.core.panel.Panel'>
Dimensions: 1 (items) x 72277 (major_axis) x 5 (minor_axis)
Items axis: BTC to BTC
Major_axis axis: 2018-02-05 17:48:00+00:00

In [2]:
def initialize(context):
    set_benchmark(symbol("BTC"))
    
def handle_data(context, data):
    
    slowma = data.history(symbol("BTC"), fields = 'price', bar_count = 50, frequency='1m').mean()
    fastma = data.history(symbol("BTC"), fields = 'price', bar_count = 10, frequency='1m').mean()
    
    if fastma < slowma:
        if symbol("BTC") not in get_open_orders():
            order_target_percent(symbol("BTC"), 0.04)
    
    if fastma > slowma:
        if symbol("BTC") not in get_open_orders():
            order_target_percent(symbol("BTC"), 0.96) 
    
    record(BTC = data.current(symbol('BTC'), fields='price'))


In [3]:
%load_ext zipline

In [4]:
import pandas as pd
from zipline.api import order, record, symbol, set_benchmark, order_target_percent, get_open_orders
import zipline
import matplotlib.pyplot as plt
from datetime import datetime

In [5]:
# C:\Users\slcf\Anaconda3\envs\py35\Lib\site-packages\zipline\utils
# C:\Users\slcf\Anaconda3\envs\py35\Lib\site-packages\trading_calendars
# https://www.zipline.io/trading-calendars.html
#https://pythonprogramming.net/custom-markets-trading-calendar-bitcoin-python-programming-for-finance/
# 새롭게 정의하고, import

#from zipline.utils import calendars
from trading_calendars.exchange_calendar_twentyfourhr import TwentyFourHR

perf = zipline.run_algorithm(start = datetime(2018, 3, 25,0,0,0,0, pytz.utc),
                            end = datetime(2018, 3, 26, 0, 0, 0, 0, pytz.utc),
                            initialize = initialize,
                             # trading calendar 바꿔주기
                            trading_calendar = TwentyFourHR(),
                            capital_base = 100000,
                            handle_data = handle_data,
                            data_frequency = 'minute',
                            data = panel)

TypeError: Can't instantiate abstract class TwentyFourHR with abstract methods close_times, open_times

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import style

style.use("ggplot")

perf.portfolio_value.pct_change().fillna(0).add(1).cumprod().sub(1).plot(label='portfolio')
perf.BTC.pct_change().fillna(0).add(1).cumprod().sub(1).plot(label='benchmark')
plt.legend(loc=2)

plt.show()